### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5986,2023-11-10,Eua Nba,21:00,Detroit Pistons,Philadelphia 76ers,3.82,1.33,220.5,1.83,1.98,6.5,2.09,1.73
5987,2023-11-10,Eua Nba,21:00,Washington Wizards,Charlotte Hornets,1.81,2.17,241.5,1.86,1.96,-3.5,2.07,1.74
5988,2023-11-10,Eua Nba,21:30,Boston Celtics,Brooklyn Nets,1.19,5.78,223.5,1.86,1.94,-11.5,1.98,1.83
5989,2023-11-10,Eua Nba,22:00,Memphis Grizzlies,Utah Jazz,1.58,2.61,229.5,1.80,1.95,-5.5,2.05,1.75
5990,2023-11-10,Eua Nba,22:00,San Antonio Spurs,Minnesota Timberwolves,3.12,1.44,224.5,1.82,1.95,1.5,2.08,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6162,2023-11-10,México Lnbp,23:00,Fuerza Regia,Halcones de Xalapa,1.22,4.15,159.5,1.83,1.91,-11.5,2.08,1.67
6163,2023-11-10,México Lnbp,23:15,Astros,Plateros,1.19,4.53,177.5,1.80,1.86,-11.5,2.00,1.72
6164,2023-11-10,México Lnbp,23:15,Libertadores,Panteras,1.29,3.51,173.5,1.88,1.88,-9.5,2.09,1.66
6165,2023-11-10,Mundo Jogos Olímpicos,19:30,Venezuela F,Porto Rico F,4.89,1.19,139.5,1.85,1.93,8.5,2.01,1.75


## Apostas do dia

### Over v0 (LDA)

In [13]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
5,22:30,Eua Nba,Dallas Mavericks,Los Angeles Clippers,230.5,1.83,0.9801,Over


### Handicap positivo V4 (LDA)

In [9]:
# Duplicando o dataset
df_hp_v4 = jogos.copy()

df_hp_v4.loc[(df_hp_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v4.loc[(df_hp_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v4['Zebra'] = df_hp_v4['Zebra'].astype('category')

# Odds
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Casa', 'Odds'] = df_hp_v4['HA_Odds_H']
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Fora', 'Odds'] = df_hp_v4['HA_Odds_A']

df_hp_v4.reset_index(drop=True, inplace=True)
df_hp_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v4.empty:
    mdl_hp_v4 = load_model('ML/handicap_positivo/v4/v4_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v4, df_hp_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
6,16:00,Europa Euroliga,Alba Berlin,Panathinaikos,2.5,2.10,1.0000,HA Casa
7,16:30,Europa Euroliga,Barcelona,Crvena zvezda,-10.5,1.71,1.0000,HA Fora
8,17:00,Europa Euroliga,Monaco,Lyon-Villeurbanne,-11.5,1.71,0.9999,HA Fora
10,10:00,Arábia Saudita Premier League,Al Salam,Al Ahli,12.5,2.00,0.5833,HA Casa
11,12:00,Arábia Saudita Premier League,Al Khaleej,Al Nasr,17.5,2.00,0.9306,HA Casa
15,20:00,Argentina Liga A,Gimnasia,Quimsa,3.5,1.97,0.9988,HA Casa
18,05:30,Austrália Liga Feminina Wnbl,Bendigo F,Sydney F,3.5,2.00,0.9899,HA Casa
25,08:35,China Cba,Beijing,Shenzhen,-6.5,1.67,0.9360,HA Fora
27,08:35,China Cba,Nanjing Tongxi,Sichuan,-12.5,1.67,0.9767,HA Fora
28,08:35,China Cba,Tianjin,Shandong,2.5,2.07,0.9936,HA Casa


### Handicap positivo V5 (RF)

In [10]:
# Duplicando o dataset
df_hp_v5 = jogos.copy()

df_hp_v5.loc[(df_hp_v5['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v5.loc[(df_hp_v5['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v5['Zebra'] = df_hp_v5['Zebra'].astype('category')

# Odds
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Casa', 'Odds'] = df_hp_v5['HA_Odds_H']
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Fora', 'Odds'] = df_hp_v5['HA_Odds_A']

df_hp_v5.reset_index(drop=True, inplace=True)
df_hp_v5.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v5.empty:
    mdl_hp_v5 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v5, df_hp_v5)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v5_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
6,16:00,Europa Euroliga,Alba Berlin,Panathinaikos,2.5,2.10,0.6800,HA Casa
7,16:30,Europa Euroliga,Barcelona,Crvena zvezda,-10.5,1.71,0.7900,HA Fora
8,17:00,Europa Euroliga,Monaco,Lyon-Villeurbanne,-11.5,1.71,0.6500,HA Fora
13,13:00,Arábia Saudita Premier League,Al-Fateh,Al Ittihad,1.5,1.83,0.6258,HA Casa
14,14:00,Arábia Saudita Premier League,Al Safa of Safwa,Al-Hilal,30.5,2.00,0.5700,HA Casa
15,20:00,Argentina Liga A,Gimnasia,Quimsa,3.5,1.97,0.7500,HA Casa
17,07:30,Austrália Nbl,Perth Wildcats,NZ Breakers,-4.5,1.76,0.5200,HA Fora
18,05:30,Austrália Liga Feminina Wnbl,Bendigo F,Sydney F,3.5,2.00,0.7800,HA Casa
19,13:00,Bósnia E Herzegovina Prvenstvo Bih,Spars Ilidza,Slavija,-10.5,1.68,0.5500,HA Fora
20,14:00,Bulgária Nbl,Levski,Shumen,-12.5,1.73,0.5200,HA Fora


### Handicap positivo V6 (LDA)

In [11]:
# Duplicando o dataset
df_hp_v6 = jogos.copy()

df_hp_v6.loc[(df_hp_v6['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v6.loc[(df_hp_v6['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v6['Zebra'] = df_hp_v6['Zebra'].astype('category')

# Odds
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Casa', 'Odds'] = df_hp_v6['HA_Odds_H']
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Fora', 'Odds'] = df_hp_v6['HA_Odds_A']

df_hp_v6.reset_index(drop=True, inplace=True)
df_hp_v6.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v6.empty:
    mdl_hp_v6 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v6, df_hp_v6)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v6_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
6,16:00,Europa Euroliga,Alba Berlin,Panathinaikos,2.5,2.10,0.6800,HA Casa
7,16:30,Europa Euroliga,Barcelona,Crvena zvezda,-10.5,1.71,0.7900,HA Fora
8,17:00,Europa Euroliga,Monaco,Lyon-Villeurbanne,-11.5,1.71,0.6500,HA Fora
13,13:00,Arábia Saudita Premier League,Al-Fateh,Al Ittihad,1.5,1.83,0.6258,HA Casa
14,14:00,Arábia Saudita Premier League,Al Safa of Safwa,Al-Hilal,30.5,2.00,0.5700,HA Casa
...,...,...,...,...,...,...,...,...
176,23:00,México Lnbp,Fuerza Regia,Halcones de Xalapa,-11.5,1.67,0.6900,HA Fora
177,23:15,México Lnbp,Astros,Plateros,-11.5,1.72,0.5500,HA Fora
178,23:15,México Lnbp,Libertadores,Panteras,-9.5,1.66,0.6800,HA Fora
179,19:30,Mundo Jogos Olímpicos,Venezuela F,Porto Rico F,8.5,2.01,0.5500,HA Casa
